## Example Two Factor ANOVA with replication

Using data from:

http://www.real-statistics.com/two-way-anova/two-factor-anova-with-replication/

Note that this implements a two factor univariate ANOVA with replication of data points. BOTH factors are fixed factors in this example.

In [8]:
import numpy as np
from scipy import special

# I'll call this class individual, so it matches the code for my real experiment. However,
# this is really the factor called "blends" in the sample.
class individual:
    def __init__(self, blend_id):
        self.blend_id = blend_id;
        self.wheat_values = np.ndarray(1)
        self.corn_values = np.ndarray(1)
        self.soy_values = np.ndarray(1)
        self.rice_values = np.ndarray(1)

    def overall_mean(self):
        all_ = np.concatenate((self.wheat_values, self.corn_values, self.soy_values, self.rice_values))
        return all_.mean()

    def overall_std(self):
        all_ = np.concatenate((self.no_dist_latencies, self.sync_latencies, self.async_latencies))
        return all_.std()

    def excludeOutlier (self, num_sds, values):
        r = 0
        while r < values.size:
            upperlimit = values.mean() + values.std()*num_sds
            lowerlimit = values.mean() - values.std()*num_sds
            if values[r] > upperlimit or values[r] < lowerlimit:
                # outlier, so delete
                values = np.delete (values, r, 0)
                r = r - 1
            r = r + 1
        return values

    def excludeOutliers (self, num_sds):
        #print "Removing outliers", num_sds, "SDs away from the mean"
        self.wheat_values = self.excludeOutlier(num_sds, self.wheat_values)
        self.corn_values = self.excludeOutlier(num_sds, self.corn_values)
        self.soy_values = self.excludeOutlier(num_sds, self.soy_values)
        self.rice_values = self.excludeOutlier(num_sds, self.rice_values)
        # Compute the sum of the squared displacements from the mean for all three conditions

    def sumofsquare_displacements_all_from_value(self, value):
        sos = self.sumofsquare_displacements_from_value(0,value) + self.sumofsquare_displacements_from_value(1,value) + self.sumofsquare_displacements_from_value(2,value) + self.sumofsquare_displacements_from_value(3,value)
        return sos

    # Compute the sum of the squared displacements from the mean for all three conditions
    def sumofsquare_displacements_all(self):
        sos = self.sumofsquare_displacements(0) + self.sumofsquare_displacements(1) + self.sumofsquare_displacements(2) + self.sumofsquare_displacements(3)
        return sos

    # Compute the sum of the squared displacements from the mean for the given condition
    def sumofsquare_displacements(self, condition):
        if condition == 0:
            mn = self.wheat_values.mean()
            squares = np.power((self.wheat_values - mn), 2)
        elif condition == 1:
            mn = self.corn_values.mean()
            squares = np.power((self.corn_values - mn), 2)
        elif condition == 2:
            mn = self.soy_values.mean()
            squares = np.power((self.soy_values - mn), 2)
        else: # condition 3
            mn = self.rice_values.mean()
            squares = np.power((self.rice_values - mn), 2)
        sos = np.sum(squares)
        return sos
    
    # Compute the sum of the squared displacements from the mean for the given condition
    def sumofsquare_displacements_from_value(self, condition, value):
        if condition == 0:
            squares = np.power((self.wheat_values - value), 2)
        elif condition == 1:
            squares = np.power((self.corn_values - value), 2)
        elif condition == 2:
            squares = np.power((self.soy_values - value), 2)
        else: # condition 3
            squares = np.power((self.rice_values - value), 2)
        sos = np.sum(squares)
        return sos

    def num_replicates_all(self):
        n = self.num_replicates(0) + self.num_replicates(1) + self.num_replicates(2) + self.num_replicates(3)
        return n

    def num_replicates(self, condition):
        if condition == 0:
            return self.wheat_values.size
        elif condition == 1:
            return self.corn_values.size
        elif condition == 2:
            return self.soy_values.size
        else: # condition 3
            return self.rice_values.size

def twofactor_anova(individuals):

    # This is a two factor anova WITH replication, as we have replicated measurements
    # (of the latency) for each individual/condition combination.
    #
    # This code follows the example in McKillup section 13.3 (p174), in which there are multiple
    # measurements for each FactorA/B combination (just as we have multiple latency readings for
    # each individual in each experimental condition)

    # Containers to fill
    wheat_values = [];
    corn_values = [];
    soy_values = [];
    rice_values = [];
    
    # Zeroth, exclude outliers:
    # In this same loop, extract the latencies into external containers:
    for ind in individuals:
        #ind = individuals[i]
        ind.excludeOutliers(2)
        wheat_values = np.concatenate((wheat_values, ind.wheat_values))
        corn_values = np.concatenate((corn_values, ind.corn_values))
        soy_values = np.concatenate((soy_values, ind.soy_values))
        rice_values = np.concatenate((rice_values, ind.rice_values))
    
    print 'wheat_values',wheat_values
    
    # Compute condition means
    wheat_mean = wheat_values.mean()
    corn_mean = corn_values.mean()
    soy_mean = soy_values.mean()
    rice_mean = rice_values.mean()

    # Compute grand mean
    all_latencies = np.concatenate((wheat_values,corn_values,soy_values,rice_values))
    grand_mean = all_latencies.mean()
    print 'GRAND MEAN:',grand_mean

    # First compute within-group variance. This means computing the sum of squares for each individual
    # and adding them all up and dividing by the relevant dof. Within-group here means "within a group
    # of latency measurements made for one individual"
    wg_sos = 0
    wg_dof = 0
    for ind in individuals:
        
        # Build up the within group sum of squares and degrees of freedom:
        sos_tmp = ind.sumofsquare_displacements_all()
        wg_sos += sos_tmp
        dof_tmp = ind.num_replicates_all()-1
        wg_dof += dof_tmp
            
    # Second, displacement due to ALL sources of variation in the experiment. Factor A, Factor B, interaction, error.
    # total_variance. This is displacement from the grand mean.
    total_sos = 0
    total_dof = -1 # dof is number of replicates -1
    for ind in individuals:
        #ind = individuals[i]
        sos_tmp = ind.sumofsquare_displacements_all_from_value(grand_mean)
        total_sos += sos_tmp
        dof_tmp = ind.num_replicates_all()
        total_dof += dof_tmp

    # So here's the total variance:
    total_variance = total_sos/total_dof
    print 'TOTAL variance:',total_variance,'sigma:',np.sqrt(total_variance),'sos:',total_sos,'dof:',total_dof

    wg_dof = (total_dof+1) - 3 * 4
    wg_variance = wg_sos/wg_dof
    print 'ERROR/WG variance:',wg_variance,'sigma:',np.sqrt(wg_variance),'sos:',wg_sos,'dof:',wg_dof

    # Now consider the data in relation to each of the two factors in turn (individual and condition).
    # These are called the "simple main effects".

    # Factor A simple main effect: Compute condition_plus_error_sos (Factor A -
    # equiv to Temperature in p177 of McKillup).
    #CROP
    wheat_treatment_mean = 0
    corn_treatment_mean = 0
    soy_treatment_mean = 0
    rice_treatment_mean = 0
    for ind in individuals:
        #ind = individuals[i]
        print 'Add ',ind.wheat_values.mean(),'to wheat_treatment_mean'
        wheat_treatment_mean += ind.wheat_values.mean()
        corn_treatment_mean += ind.corn_values.mean()
        soy_treatment_mean += ind.soy_values.mean()
        rice_treatment_mean += ind.rice_values.mean()
    
    wheat_treatment_mean = wheat_treatment_mean / len(individuals)
    corn_treatment_mean = corn_treatment_mean / len(individuals)
    soy_treatment_mean = soy_treatment_mean / len(individuals)
    rice_treatment_mean = rice_treatment_mean / len(individuals)
    
    num_replicants = 5 # in this case, num_replicants is 5 each time.
    num_rows = len(individuals) # 3 blends
    condition_plus_error_sos = num_replicants * num_rows * (np.power(wheat_treatment_mean - grand_mean,2) + np.power(corn_treatment_mean - grand_mean,2) + np.power(soy_treatment_mean - grand_mean,2) + np.power(rice_treatment_mean - grand_mean,2))
    condition_plus_error_dof = 3 # 4 conditions - 1 = 3
    
    condition_plus_error_variance = condition_plus_error_sos / condition_plus_error_dof
    print 'CONDITION variance:',condition_plus_error_variance,'sigma:',np.sqrt(condition_plus_error_variance),'sos:',condition_plus_error_sos,'dof:',condition_plus_error_dof
    
    # Factor B simple main effect. Compute individual_plus_error_sos (p178).
    # BLEND
    individual_plus_error_sos = 0
    for ind in individuals:
        #ind = individuals[i]
        ind_mean = (ind.wheat_values.mean() + ind.corn_values.mean() + ind.soy_values.mean() + ind.rice_values.mean()) / 4
        individual_plus_error_sos += np.power(ind_mean-grand_mean,2)

    num_cols = 4 # wheat, corn, soy, rice
    individual_plus_error_sos *= num_replicants * num_cols
    individual_plus_error_dof = (len(individuals)-1)
    
    individual_plus_error_variance = individual_plus_error_sos / individual_plus_error_dof
    print 'INDIVIDUAL variance:',individual_plus_error_variance,'sigma:',np.sqrt(individual_plus_error_variance),'sos:',individual_plus_error_sos,'dof:',individual_plus_error_dof

    # Alternative method to find within group degrees of freedom
    wg_dof_alt = (total_dof+1)- ((condition_plus_error_dof+1)*(individual_plus_error_dof+1))
    print 'wg_dof:',wg_dof,'wg_dof_alt:',wg_dof_alt
    
    # One way to find the interaction is by subtraction.
    interaction_sos = total_sos - condition_plus_error_sos - individual_plus_error_sos - wg_sos # McKillup p 179
    interaction_dof = condition_plus_error_dof * individual_plus_error_dof
    interaction_variance = interaction_sos / interaction_dof
    print 'INTERACTION variance (by subtraction):',interaction_variance,'sigma:',np.sqrt(interaction_variance),'sos:',interaction_sos,'dof:',interaction_dof

    # Alternatively, compute it directly
    interaction_sos2 = 0
    for ind in individuals:
        # For each condition:
        #for i in ind.wheat_values:
        interaction_sos2 += np.power((ind.wheat_values.mean() - ind.overall_mean() - wheat_mean + grand_mean),2)
        #for i in ind.corn_values:
        interaction_sos2 += np.power((ind.corn_values.mean() - ind.overall_mean() - corn_mean + grand_mean),2)
        #for i in ind.soy_values:
        interaction_sos2 += np.power((ind.soy_values.mean() - ind.overall_mean() - soy_mean + grand_mean),2)
        #for i in ind.rice_values:
        interaction_sos2 += np.power((ind.rice_values.mean() - ind.overall_mean() - rice_mean + grand_mean),2)
    interaction_sos2 *= 5
    
    interaction_variance2 = interaction_sos2 / interaction_dof
    print 'INTERACTION variance2 (by computation)):',interaction_variance2,'sigma:',np.sqrt(interaction_variance2),'sos:',interaction_sos2,'dof:',interaction_dof

    # Now compute the F ratios
    # Have wg_variance, total_variance, condition_plus_error_variance, individual_plus_error_variance and interaction_variance
    #
    # NB: When reporting must clearly state how F ratios were calculated.
    #
    
    # The website example has both factors fixed, so these are the F ratios:
    F_cond = condition_plus_error_variance / wg_variance # fixed
    F_ind = individual_plus_error_variance / wg_variance # fixed
    F_interaction = interaction_variance / wg_variance

    P_cond = 1-special.fdtr(condition_plus_error_dof,wg_dof,F_cond) # dof correct?
    P_ind = 1-special.fdtr(individual_plus_error_dof,wg_dof,F_ind)
    P_interaction = 1-special.fdtr(interaction_dof,wg_dof,F_interaction)

    #F_cond = condition_plus_error_variance / interaction_variance2 # fixed. condition + interaction + error. If both factors were fixed, this would be condition_var/error_var
    #F_ind = individual_plus_error_variance / wg_variance # random. (individual var + error var) divided by (error variance) only
    #F_interaction = interaction_variance / wg_variance
    
    # Lastly, what's the probability for this?
    #P_cond = 1-special.fdtr(condition_plus_error_dof,wg_dof+interaction_dof,F_cond) # dof correct?
    #P_ind = 1-special.fdtr(individual_plus_error_dof,wg_dof,F_ind)
    #P_interaction = 1-special.fdtr(interaction_dof,wg_dof,F_interaction)

    print '\nF_cond:{0} P_cond:{1} '.format(F_cond,P_cond)
    print 'F_ind:{0} P_ind:{1}'.format(F_ind,P_ind)
    print 'F_interaction:{0} P_interaction:{1}'.format(F_interaction,P_interaction)
    
    print '\n'
    print 'Source of var, Sumof Squares, df, Mean square , F ratio        ','P'
    print '-------------------------------------------------------------------'
    print '{0}, {1}, {2}, {3}, {4}, {5}'.format('Individual   ',individual_plus_error_sos,individual_plus_error_dof,individual_plus_error_variance,F_ind,P_ind)
    print '{0}, {1}, {2}, {3}, {4}, {5}'.format('Condition    ',condition_plus_error_sos,condition_plus_error_dof,condition_plus_error_variance,F_cond,P_cond)
    print '{0}, {1}, {2}, {3}, {4}, {5}'.format('Indiv x Cond ',interaction_sos,interaction_dof,interaction_variance2,F_interaction,P_interaction)
    print '{0}, {1}, {2}, {3}, {4}, {5}'.format('Error        ',wg_sos,wg_dof,wg_variance,'-','-')


blendX = individual('X')
blendX.wheat_values = np.array([123,156,112,100,168])
blendX.corn_values = np.array([128,150,174,116,109])
blendX.soy_values = np.array([166,178,187,153,195])
blendX.rice_values = np.array([151,125,117,155,158])

blendY = individual('Y')
blendY.wheat_values = np.array([135,130,176,120,155]) # sos: 104526
blendY.corn_values = np.array([175,132,120,187,184])
blendY.soy_values = np.array([140,145,159,131,126])
blendY.rice_values = np.array([167,183,142,167,168])

blendZ = individual('Z')
blendZ.wheat_values = np.array([156,180,147,146,193])
blendZ.corn_values = np.array([186,138,178,176,190])
blendZ.soy_values = np.array([185,206,188,165,188])
blendZ.rice_values = np.array([175,173,154,191,169])

individuals = [blendX,blendY,blendZ]

# Now actually call the two factor ANOVA
twofactor_anova(individuals)

wheat_values [ 123.  156.  112.  100.  168.  135.  130.  176.  120.  155.  156.  180.
  147.  146.  193.]
wheat_mean 146.466666667
GRAND MEAN: 157.45
TOTAL variance: 671.878813559 sigma: 25.920625254 sos: 39640.85 dof: 59
ERROR/WG variance: 442.091666667 sigma: 21.025975998 sos: 21220.4 dof: 48
Add  131.8 to wheat_treatment_mean
Add  143.2 to wheat_treatment_mean
Add  164.4 to wheat_treatment_mean
CONDITION variance: 1137.21666667 sigma: 33.7226432337 sos: 3411.65 dof: 3
INDIVIDUAL variance: 4391.45 sigma: 66.2680164182 sos: 8782.9 dof: 2
wg_dof: 48 wg_dof_alt: 48
INTERACTION variance (by subtraction): 1037.65 sigma: 32.2125751842 sos: 6225.9 dof: 6
INTERACTION variance2 (by computation)): 1037.65 sigma: 32.2125751842 sos: 6225.9 dof: 6

F_cond:2.57235490377 P_cond:0.0649438221616 
F_ind:9.93334715651 P_ind:0.000245487281371
F_interaction:2.34713765999 P_interaction:0.0455554921365


Source of var, Sumof Squares, df, Mean square , F ratio         P
-------------------------------------